# Ta feng Data

## Load all

In [1]:
from stat_data import Stat
import pickle

root_dir = "./data/ta_feng/"
filename = "transaction.dat"

f = open(root_dir+filename,"rb")
data = pickle.load(f)

f = open(root_dir+"train.pkl","rb")
train = pickle.load(f)

f = open(root_dir+"test.pkl","rb")
test = pickle.load(f)

f = open(root_dir+"validation.pkl","rb")
valid = pickle.load(f)

f = open(root_dir+"negative_sample.pkl","rb")
neg_train = pickle.load(f)

s = Stat(data)
trans = s.get_trans()
usrset = s.get_usrset()
itemset = s.get_itemset()

In [3]:
print("\nPreprosessed Data Stats")
s.show()


Preprosessed Data Stats

        # transactions: 61991
        # items       : 22326
        Ave item      : 8.6352857672888
        # users       : 10599
        


In [17]:
max_len = 0
for i in trans:
    if max_len<len(i):
        max_len=len(i)
print(max_len)

111


### Tuplize

In [3]:
if type(neg_train) is dict:
    neg_train = [(u, list(t)) for u, ts in neg_train.items() for t in ts]

In [7]:
ave = 0
for i in train:
    ave += len(i[1])
    
print(ave)

TypeError: 'int' object is not subscriptable

In [ ]:
if type(train) is dict:
    train = [(u, list(t)) for u, ts in train.items() for t in ts]

In [2]:
%load_ext autoreload
%autoreload 2

import os
import random
import pickle
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# for reproducibility
seed=1234
def seed_everything(seed=1234):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
seed_everything(seed)

In [3]:
root_dir = "./data/ta_feng/"
csv_file = "ta_feng_all_months_merged.csv"

df = pd.read_csv(root_dir+csv_file)

In [9]:
df[df["CUSTOMER_ID"]==1104905]

,TRANSACTION_DT,CUSTOMER_ID,AGE_GROUP,PIN_CODE,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE
0,11/1/2000,1104905,45-49,115,110411,4710199010372,2,24,30
50,11/1/2000,1104905,45-49,115,120103,4711524000457,1,17,21
71,11/1/2000,1104905,45-49,115,100429,4901201906015,1,32,40
94,11/1/2000,1104905,45-49,115,100201,84501297329,2,26,38
145,11/1/2000,1104905,45-49,115,110401,4710088410139,1,43,39
175,11/1/2000,1104905,45-49,115,120103,4711524000495,1,17,21
257,11/1/2000,1104905,45-49,115,100505,4710018004704,1,35,38
299,11/1/2000,1104905,45-49,115,120105,4710857000066,1,40,49
347,11/1/2000,1104905,45-49,115,100201,4710174003450,1,53,69
392,11/1/2000,1104905,45-49,115,110502,4710167221014,1,19,25


In [ ]:
customer_id = set(df["CUSTOMER_ID"])

## Change Data structure
Data structure is \
[(customer id, transaction),...]

In [ ]:
%time
import tqdm

data = []
trans = []  # for removing stop word items
n_trans = 0 # before removing fewer trans and item 
cnt = 0

for i in tqdm.tqdm(customer_id):
    customer = df[df["CUSTOMER_ID"] == i]
    trans_date = list(set(customer["TRANSACTION_DT"]))
    n_trans += len(trans_date)
    # print("custormer ID    :", i, "\n" \
    #       "transaction date:", trans_date, len(trans_date))

    for d in trans_date:
        product = customer[customer["TRANSACTION_DT"] == d]
        trans.append(list(product["PRODUCT_ID"]))
        # data.append((i, set(product["PRODUCT_ID"])))
        data.append([i, set(product["PRODUCT_ID"])])

## Transaction from 0 to 3

In [ ]:
data[0:3]

## Statics class for data stat

In [ ]:
from stat_data import Stat
s = Stat(data)

### # transactions

In [ ]:
s.n_data()

### # items

In [ ]:
itemset = s.get_itemset()
s.n_item()

### # users

In [ ]:
s.n_usr()

### Average # item in a transaction

In [ ]:
s.a_item()

---

## Preprocess

### 1. Remove stop word items

#### Count items

In [ ]:
item_dct = {item : 0 for item in itemset}
for t in trans:
    for item in t:
        item_dct[item] += 1

#### Threshold

In [ ]:
th = int(0.05*len(data))
th

In [ ]:
removed_items = {item for item, cnt in item_dct.items() if cnt>th}
print("The number of removed items:", len(removed_items), \
      "\nRemoved items \n", removed_items)

#### Remove stop word item from itemset

In [ ]:
print("Before remove:", len(itemset))
itemset -= removed_items
print("After remove :", len(itemset))

#### Remove stop word item from transaction

In [ ]:
for i in data:
    i[1] -= removed_items

### 2. Remove transactions which include less than 3 items

In [ ]:
print(f"""
Before
# Data : {len(data)}
# Items: {len(itemset)}
""")

data = [(i[0], i[1]) for i in data if len(i[1]) > 2]
itemset = set()
for i in data:
    itemset |= i[1]

print(f"""
After
# Data : {len(data)}
# Items: {len(itemset)}
""")

#### Make sure if items were removed

In [ ]:
tmp = set()
for i in data:
    tmp |= i[1]

print(len(tmp), len(itemset))
assert len(tmp)==len(itemset)
del tmp

## 3. Remove users whose transactions are fewer than 3

#### Count # transactions per user

In [ ]:
ucnt = {d[0] : 0 for d in data}

for i in data:
    ucnt[i[0]] += 1

#### Remove users

In [ ]:
data = [(d[0], d[1]) for d in data if ucnt[d[0]]>2]

### Check Stat

In [ ]:
s = Stat(data)
s.show()

## Convert list to tuple

In [ ]:
data = [(i[0], i[1]) for i in data]

## Save dataset 

In [ ]:
import pickle
root_dir = "./data/ta_feng/"
filename = "transaction.dat"

filehandler = open(root_dir+filename, "wb") 
pickle.dump(data, filehandler)

---

## Load dataset from pickle

In [1]:
import pickle
root_dir = "./data/ta_feng/"
filename = "transaction.dat"

f = open(root_dir+filename,"rb")
data = pickle.load(f)

In [2]:
from stat_data import Stat
s = Stat(data)
trans = s.get_trans()
usrset = s.get_usrset()
itemset = s.get_itemset()

## Make triple dataset for <br> BASKET-SENSITIVE FACTORIZATION MACHINE(BFM)

### Split dataset for train, test and validation based on user
Ratio is \
train : test : valid = 6 : 2 : 2

In [3]:
# dataset ratio
r_train = 0.6
r_test = 0.2
r_valid = 0.2

最初にuserがkeyでtransactionのlistがvalueの辞書型を作ってsplitする．\
(速度的に遅いかもしれない，データセットが大きくなったら遅くなるかも...)

#### Itemset per user(customer)
`usr_dct`はkeyがuserでvalueがそのuserが購入したitemの集合が入っている． \
だから，`itemset - usr_dct[usr]`でusrが購入していないアイテム集合を取得できる．これをNegative sampleを作るときに利用する．

In [4]:
import copy

usr_dct = {}
tmp = 0
for i in data:
    if not i[0] in usr_dct.keys():
         # idk why but if I don't use deepcopy, set in data is changed
        usr_dct[i[0]] = copy.deepcopy(i[1])
    else:
        usr_dct[i[0]] |= i[1]

n_usr = len(usr_dct.keys())
print(n_usr)

10599


In [5]:
udata = {usr : [] for usr in usr_dct.keys()}

for d in data:
    udata[d[0]].append(d[1])

Itemが2以下のtransactionがないかcheck

In [6]:
cnt = 0
for u, t in udata.items():
    if len(t) < 3: 
        cnt+=1
assert cnt==0, "Invalid transaction"

### Split based on user

In [7]:
import copy
import math

# Original transaactions per user
orig_ts = {usr:[] for usr in udata.keys()}
# Copy same dict structure
train = copy.deepcopy(orig_ts)
test = copy.deepcopy(orig_ts)
valid = copy.deepcopy(orig_ts)

for u,t in udata.items():
    l_t = len(t)
    if l_t == 3:
        train[u] = [copy.deepcopy(t[0])]
        test[u] = [copy.deepcopy(t[1])]
        valid[u] = [copy.deepcopy(t[2])]
    else:
        l_train = math.ceil(l_t*r_train)

        train[u] = copy.deepcopy(t[:l_train])

        l_test = math.ceil((l_t-l_train)*0.5)
        test[u] = copy.deepcopy(t[l_train:l_train+l_test])

        valid[u] = copy.deepcopy(t[l_train+l_test:])

calculate sum of data

In [8]:
cnt = 0

for tr, te, va in zip(train.values(), test.values(), valid.values()):
    cnt += (len(tr) + len(te) + len(va))

assert cnt==len(data), f"Invalid split, before split: {len(data)}, after split: {cnt}"

### Calculate pmi
We only need pmi for train dataset, so we calculated train dataset pmi with original dataset(valiable is `data`).

### Make negative sample
Negative sampleはPositive sampleをベースに作られている．\
1つのPositive sampleにつき2つ作る．つまりNegative sampleはPositive sampleの数の2倍になる．\
Negative sampleはベースとなっているアイテム数(|B|+v)と同じになっている．\
Negative sampleのアイテムはそのUserが購入していないアイテムで構成される．

In [19]:
import tqdm
import random
random.seed(1237)

def make_neg(src, usr_dct, udata, rate=1):
    neg = {usr:[] for usr in udata.keys()}
    for usr, trans in tqdm.tqdm(src.items()):
        for t in trans:
            # item length
            l_item = len(t)

            if rate==1:
                # 1* negative itemset 
                neg_itemset = itemset - usr_dct[usr]
                neg_data = random.sample(neg_itemset, k=l_item)
                neg[usr].append(set(neg_data[:l_item]))
            elif rate==2:
                # 2* negative itemset 
                neg_itemset = itemset - usr_dct[usr]
                neg_data = random.sample(neg_itemset, k=l_item*2)
                neg[usr].append(set(neg_data[:l_item]))
                neg[usr].append(set(neg_data[l_item:]))
            else:
                print("Invalid rate. Rate 1 or 2 is implemented.")
                return None
                
    # Test length
    l_src = 0
    for trans in src.values():
        l_src += len(trans)

    l_neg = 0
    for trans in neg.values():
        l_neg += len(trans)

    assert rate*l_src==l_neg, \
           f"""The number of negative sample should be 2*src.
                    But got train sample:{l_src}, negative sample:{l_neg}"""

    return neg

rate=2
neg_train = make_neg(src=train, usr_dct=usr_dct, udata=udata, rate=rate)
neg_test = make_neg(src=test, usr_dct=usr_dct, udata=udata, rate=rate)
neg_valid = make_neg(src=valid, usr_dct=usr_dct, udata=udata, rate=rate)

100%|██████████| 10599/10599 [00:08<00:00, 1208.35it/s]


### Save negative sample

In [20]:
if rate==1 or rate==2:
    filehandler = open(root_dir+f"neg/neg_train_{rate}.pkl", "wb") 
    pickle.dump(neg_train, filehandler)

    filehandler = open(root_dir+f"neg/neg_test_{rate}.pkl", "wb") 
    pickle.dump(neg_test, filehandler)

    filehandler = open(root_dir+f"neg/neg_valid_{rate}.pkl", "wb") 
    pickle.dump(neg_valid, filehandler)
else:
    print("Invalid rate. Rate 1 or 2 is implemented.")

### Save train, test, validation and negative sample dataset

In [ ]:
if flg:
    filehandler = open(root_dir+"train.pkl", "wb") 
    pickle.dump(train, filehandler)

    filehandler = open(root_dir+"test.pkl", "wb") 
    pickle.dump(test, filehandler)

    filehandler = open(root_dir+"validation.pkl", "wb") 
    pickle.dump(valid, filehandler)

    filehandler = open(root_dir+"negative_sample.pkl", "wb") 
    pickle.dump(neg_train, filehandler)
else:
    print("Flag is False")

### Load train, test, validation and negative sample

In [1]:
import pickle
root_dir = "./data/ta_feng/"

f = open(root_dir+"train.pkl","rb")
train = pickle.load(f)

f = open(root_dir+"test.pkl","rb")
test = pickle.load(f)

f = open(root_dir+"validation.pkl","rb")
valid = pickle.load(f)

f = open(root_dir+"negative_sample.pkl","rb")
neg_train = pickle.load(f)

## Convert data for model input using pytorch

In [ ]:
import torch
import torch.nn.functional as F

Convert Itemset

In [ ]:
itemset = list(itemset)
itemset.sort()
print(itemset.index(itemset[0]))

o_item = F.one_hot(torch.arange(0, len(itemset)))

Convert User

In [ ]:
usrset = list(usrset)
usrset.sort()
print(usrset.index(usrset[0]))

o_usr = F.one_hot(torch.arange(0, len(usrset)))

In [ ]:
o_item

In [ ]:
o_usr

In [ ]:
a = o_item[itemset.index(train[0][1][0])]
print("Tensor :", a)
print("Index  :", (a == 1).nonzero())

### Convert train data structure to input format

#### <span style="color: red; ">Too much memory is used</span><pre>So, I have to use yeild to make generator.

In [ ]:
len(train), type(train)

In [ ]:
random.Random(seed).sample(train, len(train))[:5]

In [ ]:
def convert(train, usrset, itemset, o_usr, o_item):
    train_input = []

    for d in train:
        usr = d[0]
        trans = d[1]
        basket = 0

        usr = o_usr[usrset.index(usr)]

        for item in trans:
            basket += o_item[itemset.index(item)]

        # print(trans)
        # print("items:", len(trans), basket.sum())
        # print("general basket\n", (basket==1).nonzero())
        for item in trans:
            # print(item)
            target = o_item[itemset.index(item)]
            t = basket-target
            # print("target \n", (target==1).nonzero())
            # print("t nonzero\n", (t==1).nonzero())
            t = torch.cat((target, t))
            # print(target.sum(), t[:target.shape[0]].sum(), t[target.shape[0]:].sum())
            t = torch.cat((usr, t))
            # train_input.append(t)
            yield t

In [ ]:
tmp = convert(train, usrset, itemset, o_usr, o_item)

t = next(tmp)
print(t, (t==1).nonzero())
t = next(tmp)
print(t, (t==1).nonzero())
t = next(tmp)
print(t, (t==1).nonzero())
t = next(tmp)
print(t, (t==1).nonzero())


In [ ]:
%%time

import torch
import torch.nn.functional as F

def usr2vec(usrset):
    usrset = list(usrset)
    usrset.sort()
    o_usr = F.one_hot(torch.arange(0, len(usrset)))
    
    return usrset, o_usr

def item2vec(itemset):
    itemset = list(itemset)
    itemset.sort()
    o_item = F.one_hot(torch.arange(0, len(itemset)))
    
    return itemset, o_item

if type(train) is dict:
    train = [(u, list(t)) for u, ts in train.items() for t in ts]
    
usrset, o_usr = usr2vec(usrset)
itemset, o_item = item2vec(itemset)
ds = convert(train, usrset, itemset, o_usr, o_item)

cnt = 0
for d in ds:
    cnt += 1

print(cnt)
"""
    print(d)
    print("Index  :", (d == 1).nonzero())
    cnt += 1
    if cnt==4:
        break
"""

## Make $t^m$ transaction

In [139]:
import pickle
f = open(root_dir+"train.pkl","rb")
train = pickle.load(f)
 
# All transactions
all_trans = [t for ts in train.values() for t in ts]

# Maybe it is not correct
cnt=0
# for trans in train.values():
for user, trans in train.items():
    for i, t_1 in enumerate(trans):
        for t_2 in trans[i+1:]:
            if t_1==t_2:
                print(user)
                print(f"t_1: {t_1}")
                print(f"t_2: {t_2}")
                cnt+=1
print(cnt) 

2109190
t_1: {4711856020208, 4710085104130, 4711856020215}
t_2: {4711856020208, 4710085104130, 4711856020215}
2006031
t_1: {4710094097768, 4710579101003, 4710085104116, 4710094699078}
t_2: {4710094097768, 4710579101003, 4710085104116, 4710094699078}
2034515
t_1: {4710043231410, 9318017209059, 4710043000429}
t_2: {4710043231410, 9318017209059, 4710043000429}
729031
t_1: {4711985005626, 4710883000092, 4711209051026}
t_2: {4711985005626, 4710883000092, 4711209051026}
729031
t_1: {4711985005626, 4710883000092, 4711209051026}
t_2: {4711209051026, 4711985005626, 4710883000092}
729031
t_1: {4711985005626, 4710883000092, 4711209051026}
t_2: {4711209051026, 4711985005626, 4710883000092}
239448
t_1: {4710320153121, 4711080040195, 4710105015125}
t_2: {4710320153121, 4711080040195, 4710105015125}
906456
t_1: {20415723, 4710719000821, 4710583350701}
t_2: {20415723, 4710719000821, 4710583350701}
8


In [167]:
import pickle
import numpy as np

f = open(root_dir+"train.pkl","rb")
train = pickle.load(f)
 
# All transactions
all_trans = [t for ts in train.values() for t in ts]

tran = list(train.values())[0][0]
tran = list(tran)
print(tran)
print(len(tran))

result = []
for i, v_1 in enumerate(tran):
    subtran = tran[i+1:]
    if len(subtran)==0:
        continue
    cv_1=0
    v_1_result = []
    tmp = []
    for v_2 in subtran:
        cv_12=0
        cv_2=0
        for t in all_trans:
            if v_1 in t:
                cv_1+=1
            if v_2 in t:
                cv_2+=1
            if v_1 in t and v_2 in t:
                cv_12+=1
        pmi = np.log(cv_12/(cv_1*cv_2))
        result.append((v_1, v_2, pmi))
        tmp.append(pmi)
    print(max(tmp))
    v_1_result.append(max(tmp))
        
"""
print("v_1, v_2, pmi")
for presult in result:
    print(presult)
print(len(result))
"""

[4710431397056, 4710015102946, 4715833014310, 4710431330282, 4710431324236, 4715833010893, 4714005052013, 4713691062900, 4715190002999]
9
-5.472270673671475
-5.529429087511423
-4.499809670330265
-4.882801922586371
-4.276666119016055
-3.1780538303479458
-3.7376696182833684
-1.3862943611198906
[-1.3862943611198906]


'\nprint("v_1, v_2, pmi")\nfor presult in result:\n    print(presult)\nprint(len(result))\n'

In [62]:
# Have to find a transaction that has the maximum pmi score.
def pmi(user_ts):
    """t_m is t^m transaction"""
    trans = list(user_ts.items())[0][1]
    co_matrix = []
    for num, t1 in enumerate(trans):
        v1 = 
        for t2 in trans[num:]:
            for i in t:
                # matrix
            
    return t_mv_1_result

SyntaxError: invalid syntax (<ipython-input-62-e573577e7887>, line 6)

---
---
---
---

Preprocessing when store data

In [ ]:
%time
data = []
trans = []  # for removing stop word items
n_trans = 0 # before removing fewer trans and item 
cnt = 0

for i in customer_id:
    customer = df[df["CUSTOMER_ID"] == i]
    trans_date = list(set(customer["TRANSACTION_DT"]))
    n_trans += len(trans_date)
    # print("custormer ID    :", i, "\n" \
    #       "transaction date:", trans_date, len(trans_date))

    # Remove users whose transactions are fewer than 3.
    if len(trans_date) < 4:
        cnt += 1
        continue

    for d in trans_date:
        product = customer[customer["TRANSACTION_DT"] == d]
        
        # Store only transactions which include more than 2 items.
        if len(set(product["PRODUCT_ID"])) > 2:
            trans.append(list(product["PRODUCT_ID"]))
            # data.append((i, set(product["PRODUCT_ID"])))
            data.append([i, set(product["PRODUCT_ID"])])
        else:
            cnt += 1

## split data using sklearn func (not based on user)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
seed=1234

# Shuffle data
data = shuffle(data, random_state=seed)

# Split train, test and validation
train, test = train_test_split(data, test_size=0.4, random_state=seed)
test, valid = train_test_split(test, test_size=0.5, random_state=seed)

In [ ]:
len(train), len(test), len(valid)

---
---
---
---

#### Adjacency matrix

In [ ]:
import tqdm

def make_adj(num_items, itemset, transactions):
    adjMatrix = np.zeros((num_items, num_items), dtype=np.float32)
    for t in tqdm.tqdm(transactions):
        for i, item1 in enumerate(t):
            # item1がtrain_itemsetに入ってなかったら飛ばす
            if not item1 in itemset: break
                
            sub_t = t[i+1:] # こっちはただの隣接行列
            idx1 = itemset.index(item1)
            for item2 in sub_t:
                # item2がtrain_itemsetに入ってなかったら飛ばす
                if not item2 in itemset: break
                idx2 = itemset.index(item2)
                adjMatrix[idx1][idx2] += 1

    return adjMatrix.astype(np.int32)

##### Train itemset

In [ ]:
train_itemset = set()
for i in train:
    train_itemset |= i[1]

train_itemset = list(train_itemset)
print(len(train_itemset))
n_train_item = len(train_itemset)

##### Whole transaction is `trans`

In [ ]:
adj = make_adj(n_train_item, train_itemset, trans)

##### Save adjacency matrix as numpy object

In [ ]:
print(adj)
np.save(f"{root_dir}adj", adj)

In [ ]:
train[0]

In [ ]:
# 内包表記のほうが速いっぽい
# ref: https://utgwkk.hateblo.jp/entry/2017/03/09/154314

# custormer id == 1835021
%time
custormer_id = 1835021
# target_custormer = lambda x: x[0]==custormer_id
# u_data = map(lambda x: x[1] if x[0]==custormer_id else None, trans_from_pickle)
# u_data = list(filter(target_custormer, trans_from_pickle))

In [ ]:
%time
target_tran = [i[1] for i in data if i[0]==custormer_id]

In [ ]:
target_tran